In [8]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
# Use ensemble to train the model
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd
import lightgbm as lgb
import sys
import sys

# In the second part of the notebook
from sklearn.preprocessing import LabelEncoder

# This part of importing would be moved to main.py or train.py later
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

from scipy.stats import zscore

import pandas as pd
from sklearn.feature_selection import chi2
from sklearn.preprocessing import LabelEncoder
# from catboost import CatBoostRegressor

import lightgbm as lgb
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

import lightgbm as lgb
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import wandb

Define a hyperparameter class 

In [ ]:
class Config:
    def __init__(self):
        self.objective = 'regression'
        self.num_leaves = 31
        self.learning_rate = 0.01
        self.n_estimators = 120
        self.max_bin = 255
        self.bagging_fraction = 0.8
        self.bagging_freq = 5
        self.feature_fraction = 0.8
        self.feature_fraction_seed = 9
        self.bagging_seed = 9
        self.random_state = 42
        self.OrgFertilizers = 'None'
        self.FirstTopDressFert = 'None' 
        self.CropbasalFerts = 'None'
        self.NursDetFactor = 'None'
        self.LandPreparationMethod = 'None'
        self.TransDetFactor = 'None'
        self.PCropSolidOrgFertAppMethod = 'None'
        self.sparse_threshold = 1.0
# Instantiate the Hyperparameters class
config = Config()

# Initialize a wandb run
wandb.init(project='zindi-crop-challenge',
           entity="lmu-seminar")

# Update the wandb configuration with the hyperparameters
wandb.config.update(vars(config))
    

In [ ]:
# Initialize a new wandb run
wandb.init(project='zindi-crop-challenge', 
           entity="lmu-seminar")

# Define your hyperparameters
hyperparameters = dict(
    objective='regression',
    num_leaves=31,
    learning_rate=0.009,
    n_estimators=120,
    # Add other hyperparameters you want to track here
)

# Save hyperparameters to wandb
wandb.config.update(hyperparameters)

# Rest of your code for preparing the data
# ...

# Parameters for cross-validation
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=82)

rmse_scores = []
models = []

feature_importances = np.zeros(X_train.shape[1])

# Start the K-Fold cross-validation loop
for fold, (train_index, val_index) in enumerate(kf.split(X_train)):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    # Create a LGBMRegressor object with hyperparameters from wandb
    lgbm_model = lgb.LGBMRegressor(**wandb.config)
    
    # Train the model
    lgbm_model.fit(
        X_train_fold, y_train_fold, 
        eval_set=[(X_val_fold, y_val_fold)], 
        eval_metric='mae', 
        categorical_feature=whole_categorical_cols,
        callbacks=[wandb.callback()]
    )
    
    # Predict on the validation set
    y_pred_val = lgbm_model.predict(X_val_fold, num_iteration=lgbm_model.best_iteration_)

    # Calculate RMSE and log the metric to wandb
    fold_rmse = np.sqrt(mean_squared_error(y_val_fold, y_pred_val))
    rmse_scores.append(fold_rmse)
    wandb.log({'fold_rmse': fold_rmse, 'fold': fold})

    # Feature importances
    feature_importances += lgbm_model.feature_importances_

    # Save the model to wandb
    wandb.sklearn.log_model(lgbm_model, 'LGBMRegressor')
    
    models.append(lgbm_model)

# Log mean and standard deviation of RMSE across folds to wandb
wandb.log({'mean_rmse': np.mean(rmse_scores), 'std_rmse': np.std(rmse_scores)})

# Feature importances
feature_importances = feature_importances / n_splits
# You can also log feature importances to wandb here

# Finalize and close your wandb run
wandb.finish()


In [ ]:
class Hyperparameters:
    objective = 'regression'
    num_leaves = 31
    learning_rate = 0.009
    n_estimators = 120
     


In [9]:
# Read the dataset
path = r'C:\Users\ra78lof\Not-so-auto-ml\Train.csv'
dataset = pd.read_csv(path)

In [10]:
test_path = r'C:\Users\ra78lof\Not-so-auto-ml\Test.csv'
dataset_test = pd.read_csv(test_path)

In [11]:
# Save the 'Yield' column
train_labels = dataset['Yield'].copy()
dataset = dataset.drop(columns=['Yield'])

# Copy the original dataset
dataset_original = dataset.copy()
dataset_test_original = dataset_test.copy()

In [12]:
for col in dataset.columns:
    if dataset[col].dtype == 'object':
        dataset[col] = dataset[col].astype('category').cat.codes
    elif dataset[col].dtype == 'datetime64[ns]':
        dataset = dataset.drop(columns=[col])

for col in dataset_test.columns:
    if dataset_test[col].dtype == 'object':
        dataset_test[col] = dataset_test[col].astype('category').cat.codes
    elif dataset_test[col].dtype == 'datetime64[ns]':
        dataset_test = dataset_test.drop(columns=[col])

In [13]:
X = dataset
y = train_labels    

# Train Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=200)

# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Select the object columns and categorical columns
object_cols = X_train.select_dtypes(include=['object'])
categorical_cols = X_train.select_dtypes(include=['category'])
whole_categorical_cols = pd.concat([object_cols, categorical_cols], axis=1) 
# generate a list with the name of the categorical columns
whole_categorical_cols = whole_categorical_cols.columns
# Transfer into a list
whole_categorical_cols = list(whole_categorical_cols)

In [14]:
# Parameters
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=82)

# Prepare an array to store the RMSE for each fold
rmse_scores = []
models = []

# Initialize an empty array to hold feature importances
feature_importances = np.zeros(X_train.shape[1])

# Start the K-Fold cross-validation loop
for fold, (train_index, val_index) in enumerate(kf.split(X_train)):
    # Split the data
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    # Create a LGBMRegressor object
    lgbm_model = lgb.LGBMRegressor(objective='regression', num_leaves=31, learning_rate=0.01, n_estimators=100)
    
    # Train the model
    lgbm_model.fit(
        X_train_fold, y_train_fold, 
        eval_set=[(X_val_fold, y_val_fold)], 
        eval_metric='mae', 
        categorical_feature=whole_categorical_cols
    )
    
    # Predict on the validation set
    y_pred_val = lgbm_model.predict(X_val_fold, num_iteration=lgbm_model.best_iteration_)

    # Calculate and print RMSE for the current fold
    fold_rmse = np.sqrt(mean_squared_error(y_val_fold, y_pred_val))
    rmse_scores.append(fold_rmse)
    print(f"Fold {fold}: RMSE: {fold_rmse}")

    # Accumulate feature importances
    feature_importances += lgbm_model.feature_importances_

    models.append(lgbm_model)

# After cross-validation, print the mean RMSE
print(f"Mean RMSE: {np.mean(rmse_scores)}, STD RMSE: {np.std(rmse_scores)}")

# Feature importances from all folds
feature_importances = feature_importances / n_splits

test_predictions = []

for model in models:
    # Make predictions
    fold_preds = model.predict(dataset_test, num_iteration=model.best_iteration_)
    test_predictions.append(fold_preds)

# Now average these predictions
test_predictions = np.column_stack(test_predictions)
y_pred_test = np.mean(test_predictions, axis=1)

# Now you have `y_pred_test` which is the averaged predictions from all folds

# Create a submission file
dataset_upload = dataset_test_original

submission_df = pd.DataFrame({'ID': dataset_upload['ID'], 'Yield': y_pred_test})
submission_df.to_csv('submission_07_11_567.csv', index=False)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000472 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1658
[LightGBM] [Info] Number of data points in the train set: 2786, number of used features: 42
[LightGBM] [Info] Start training from score 599.349605
Fold 0: RMSE: 347.1344370327278
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000386 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1683
[LightGBM] [Info] Number of data points in the train set: 2786, number of used features: 42
[LightGBM] [Info] Start training from score 594.891960
Fold 1: RMSE: 412.0191218418146
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000434 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1666
[LightGBM] [Info] Number of data points in the train s